# Problem Set 6:

## Neutrino oscillations

The flavour mixing matrix for the three neutrino families is


$$
\begin{pmatrix}
   \nu_{e} \\
   \nu_{\mu}  \\
   \nu_{\tau}
\end{pmatrix}
=
\begin{pmatrix}
   U_{11} & U_{12} & U_{13} \\
   U_{21} & U_{22} & U_{23} \\
   U_{31} & U_{32} & U_{33}
\end{pmatrix}
\cdot
\begin{pmatrix}
   \nu_{1} \\
   \nu_{2}  \\
   \nu_{3}
\end{pmatrix}
$$

with $U$ (assuming CP phase $\delta$ = 0),

$$
U=
\begin{pmatrix}
   c_{12}c_{13} & s_{12}c_{13} & s_{13} \\
   -c_{12}s_{13}s_{23} - s_{12}c_{23} & -s_{12}s_{13}s_{23} + c_{12}c_{23} & c_{13}s_{23} \\
   -c_{12}s_{13}c_{23} + s_{12}s_{23} & -s_{12}s_{13}c_{23} - c_{12}s_{23} & c_{13}c_{23}
\end{pmatrix}
.
$$

The probability of transition of a neutrino of flavour $\alpha$ to $ \beta$ is given by:


$$
\mathcal{P}_{\nu_{a} \rightarrow \nu_{b}} (L, E) = \delta_{ab} + \sum_{i=1}^{3}\sum_{j=i+1}^{3} U_{a i} U_{b i} U_{a j} U_{b j} \sin^2 \bigg( \frac{\Delta m^2_{ij} L}{4E} \bigg) 
$$


Derive the expression of the survival probability of an electronic neutrino.
Compare with the result obtained in the problem set 4. We will now graphically compare the two in this notebook. 

The flavour mixing matrix is constructed using Numpy arrays through the PMNS function (see *utilities.py* for details) which takes as input $\theta_{12}$, $\theta_{13}$ and $\theta_{23}$. Currently $\theta_{13}$ is set to 0.

In [1]:
import numpy as np
from utilities import PMNS


t12 = np.arcsin((1/3)**0.5)
t13 = np.arcsin(0.)
t23 = np.arcsin(0.5**0.5)

U = PMNS(t12, t13, t23)
U

array([[ 0.81649658,  0.57735027,  0.        ],
       [-0.40824829,  0.57735027,  0.70710678],
       [ 0.40824829, -0.57735027,  0.70710678]])

you can check that the matrix is unitary

In [2]:
U.dot(np.conj(U.T))

array([[ 1.00000000e+00,  0.00000000e+00, -5.55111512e-17],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-5.55111512e-17,  0.00000000e+00,  1.00000000e+00]])

The probability of the transition $\nu_{a} \rightarrow \nu_{b}$ is computed with the function *posc* which takes as inputs the flavour of **a** and **b** (electron = 0, muon = 1, tau = 2) the mass differences between the mass eigenstates and values for L/E.

In [3]:
from hepunits.constants import c_light, h_Planck, hbar
from hepunits.units import GeV, km, eV, MeV


# mass differences between mass eigenstates
d21 = 7.53E-05 # in eV^2
d32 = 2.51E-03
d31 = d32 + d21

dm2 = [d21, d31, d32]

def prob_survival_exo4(d21, t12, LEratio):
    """
    Gives the survival probability for nu(e) -> nu(e) in the two flavour mixing case
    L in km and E in GeV, and dm2 in eV^2
    """
    
    return 1 - np.sin(2*t12)**2 * np.sin(1.27*d21 * LEratio)**2

def posc(a, b, U, dm2, LEratio):
    """
    Gives the oscillation probability for nu(a) -> nu(b)
    for PMNS matrix U, and L in km and E in GeV, and dm2 in eV^2
    """
    
    LE = (LEratio * km) / (4 * hbar * c_light * GeV)
    
    s = 0
    for i in range(2):
        for j in range(i+1, 3): 
            _dm2_ = dm2[i+j-1] * eV**2
            arg = _dm2_ * LE
            mxe = U[a,i]*U[b,i]*U[a,j]*U[b,j]
            s += -4*mxe*np.sin(arg)**2
    if a == b: s += 1.0
    return s

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [5]:
mpl.rcParams["figure.figsize"] = (10, 6)
mpl.rcParams["legend.fontsize"] = 14

Plot of the survival probability of $\nu_{e}$ produced 180 km away in fonction of the neutrino energy:

In [6]:
from utilities import plot_compare, interactive_plot

In [7]:
L = 180 # km
E = np.linspace(1, 100, 10000) * MeV #in MeV
E = E/GeV #in GeV

In [8]:
interactive_plot(plot_compare(prob_survival_exo4, posc), L, E, log=True)

interactive(children=(FloatSlider(value=0.0, description='t13', max=0.2, step=0.001), Checkbox(value=True, des…

What do you observe if you change the value of $\theta_{13}$ (Don't hesitate also to play with the other parameters).

You can also plot the probability of oscillations of the electrons into other flavour in function of L/E: 

In [9]:
from utilities import plot_posc
bounds = (0, 36000)
LE = np.linspace(*bounds, 3600)

In [10]:
from utilities import plot_posc
bounds = (0, 36000)
LE = np.linspace(*bounds, 3600)
interactive_plot(plot_posc(posc, bounds, "L/E [km/GeV]"), LE);

interactive(children=(FloatSlider(value=0.0, description='t13', max=0.2, step=0.001), Checkbox(value=False, de…

What can you say about $\mathcal{P}_{\nu_{e} \rightarrow \nu_{\mu}}$ and $\mathcal{P}_{\nu_{e} \rightarrow \nu_{\tau}}$ when $\theta_{13}$ = 0 and $\theta_{13}$ > 0 ?

In [11]:
bounds = (1000, 100000)
LE = np.linspace(*bounds, 10000)
interactive_plot(plot_posc(posc, bounds, "L/E [km/GeV]"), LE, log=True);

interactive(children=(FloatSlider(value=0.0, description='t13', max=0.2, step=0.001), Checkbox(value=True, des…

### Effect of the resolution on L and E:

in a real experiment E and L have a spread that can affect the detection of the oscillations. To take into account the resolutions on L/E the oscillation probability is convoluted with a Gaussian distribution of mean L/E and a width which is the resolution.

In [12]:
def resolution(LE):
    #return 10e15
    return 0.2 * LE

In [13]:
def posc_conv(a, b, U, dm2, LEratio, resolution):
    """
    Gives the oscillation probability for nu(a) -> nu(b)
    for PMNS matrix U, and L in km and E in GeV, and dm2 in eV^2
    """
    
    LE = (LEratio * km) / (4 * hbar * c_light * GeV)
    sigma_LE = resolution(LE)
    
    s = 0
    for i in range(2):
        for j in range(i+1, 3):
            _dm2_ = dm2[i+j-1] * eV**2
            arg = _dm2_ * LE
            mxe = U[a,i]*U[b,i]*U[a,j]*U[b,j]
            s += -2 * mxe * ( 1 - np.cos(2 * arg) * np.exp(-2*_dm2_**2*sigma_LE**2) )
    if a == b: s += 1.0
    return s

A new *posc_conv* function has been created which is the same as *posc* but takes an additionnal resolution function. We know plot on the same graph the $\nu{e}$ survival probability with and without the resolution (don't hesitate to play with other resolution models).

In [14]:
from utilities import plot_posc_conv
xbounds = (100, 200000)
LE = np.linspace(*xbounds, 100000)
interactive_plot(plot_posc_conv(posc, posc_conv, xbounds, "L/E [km/GeV]", resolution=resolution), LE, log=True);

interactive(children=(FloatSlider(value=0.0, description='t13', max=0.2, step=0.001), Checkbox(value=True, des…

## Measurement of $\theta_{13}$:

The electronic neutrino survival probability oscillates on two different lenght scales. The short wavelenght component, which depends on $\Delta m^2_{32}$, oscillates with an amplitude of $\sin^2(2\theta_{13})$ and the longer wavelenght component depends on $\Delta m^2_{12}$. 
Knowing values of the mass eigenstates differences,

$$\Delta m^2_{21} = 7.53\times 10^{-5} \text{ eV}^2$$ 
$$\Delta m^2_{32} = 2.51\times 10^{-3} \text{ eV}^2$$

the measurements of the $\nu_{e}$ survival pobability at distances $\mathcal{O}(1)$ km are sensitive to $\theta_{13}$ and measurements at distances of $\mathcal{O}(100)$ are sensitive to $\Delta m^2_{12}$ and $\theta_{12}$ as can been see in the following plot.

In [15]:
xbounds = (0, 150)
L = np.linspace(*xbounds, 100000)
E = 3 * (MeV / GeV)
interactive_plot(plot_posc(posc, xbounds, "L [km]", x=L), L, E);

interactive(children=(FloatSlider(value=0.0, description='t13', max=0.2, step=0.001), Checkbox(value=False, de…

The Daya Baya experiment was the first to observe a non zero value for $\theta_{13}$ in 2000. It's a reactor experiment producing $\bar{\nu_{e}}$ for $\beta$-decays of radioisotopes such as $^{235}$U, $^{238}$U, $^{239}$Pu and $^{241}$Pu, which are produced in nuclear fission. The mean energy of the reactor antineutrinos is about 3 MeV. 
The experiment consists of six detector, two at a mean at a distance of 470 m from the reactors, one at 576 m and three at 1.65 km.

In [16]:
E = 3 * (MeV / GeV) # MeV
xbounds = (0, 4)
L = np.linspace(*xbounds, 1000) #Km
interactive_plot(plot_posc_conv(posc, posc_conv, xbounds, "L [km]", resolution=resolution, x=L,
                                ybounds=(0.88, 1.025), t13=0.16, dayabay=True), L, E, log=False);

interactive(children=(FloatSlider(value=0.16, description='t13', max=0.2, step=0.001), Checkbox(value=False, d…